In [0]:
student = [(1, 'Shiva'),(2, 'Pavan'),(3, 'Vishwa'), (4, 'Aditya')]
stu_schema = "student_id int, Student_name string"

marks = [(1, 'PySpark', 90),(1, 'SQL', 100),
         (2, 'PySpark', 80),(2, 'SQL', 30),
         (3, 'PySpark', 20),(3, 'SQL', 20),
         (4, 'PySpark', 90),(4, 'SQL', 90)]
marks_schema = "student_id int, Subject_name string, marks int"

student_df = spark.createDataFrame(student, stu_schema)
marks_df = spark.createDataFrame(marks, marks_schema)
student_df.show()
marks_df.show()

+----------+------------+
|student_id|Student_name|
+----------+------------+
|         1|       Shiva|
|         2|       Pavan|
|         3|      Vishwa|
|         4|      Aditya|
+----------+------------+

+----------+------------+-----+
|student_id|Subject_name|marks|
+----------+------------+-----+
|         1|     PySpark|   90|
|         1|         SQL|  100|
|         2|     PySpark|   80|
|         2|         SQL|   30|
|         3|     PySpark|   20|
|         3|         SQL|   20|
|         4|     PySpark|   90|
|         4|         SQL|   90|
+----------+------------+-----+



In [0]:
from pyspark.sql.functions import col,sum,count, when

In [0]:
marks_df1= marks_df.groupBy(col('student_id')).agg(
    sum(col('marks')).alias('total_marks'),
    count(col('Subject_name')).alias('total_subjects')
    )
marks_df1.show()

+----------+-----------+--------------+
|student_id|total_marks|total_subjects|
+----------+-----------+--------------+
|         1|        190|             2|
|         2|        110|             2|
|         3|         40|             2|
|         4|        180|             2|
+----------+-----------+--------------+



In [0]:
marks_df2 = marks_df1.withColumn("percentage", (col("total_marks")*100.0)/(col("total_subjects")*100))
marks_df2.show()

+----------+-----------+--------------+----------+
|student_id|total_marks|total_subjects|percentage|
+----------+-----------+--------------+----------+
|         1|        190|             2|      95.0|
|         2|        110|             2|      55.0|
|         3|         40|             2|      20.0|
|         4|        180|             2|      90.0|
+----------+-----------+--------------+----------+



In [0]:
marks_df3 = marks_df2.withColumn("Class",
                                 when(col("percentage") >= 70, 'Distinction')\
                                    .when(col("percentage").between(60,69), 'First_Class')\
                                        .when(col("percentage").between(50,59), 'Second_Class').otherwise("Fail")
)

In [0]:
marks_df3.show()

+----------+-----------+--------------+----------+------------+
|student_id|total_marks|total_subjects|percentage|       Class|
+----------+-----------+--------------+----------+------------+
|         1|        190|             2|      95.0| Distinction|
|         2|        110|             2|      55.0|Second_Class|
|         3|         40|             2|      20.0|        Fail|
|         4|        180|             2|      90.0| Distinction|
+----------+-----------+--------------+----------+------------+



In [0]:
final_df = marks_df3.join(student_df, on = "student_id").select(student_df.student_id, student_df.Student_name, marks_df3.percentage, marks_df3.Class)

final_df.show()

+----------+------------+----------+------------+
|student_id|Student_name|percentage|       Class|
+----------+------------+----------+------------+
|         1|       Shiva|      95.0| Distinction|
|         2|       Pavan|      55.0|Second_Class|
|         3|      Vishwa|      20.0|        Fail|
|         4|      Aditya|      90.0| Distinction|
+----------+------------+----------+------------+



##Question2

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType

In [0]:
schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("emp_name", StringType(), True),
    StructField("emp_gender", StringType(), True),
    StructField("emp_age", IntegerType(), True),
    StructField("emp_salary", IntegerType(), True),
    StructField("emp_manager", StringType(), True)
])

In [0]:
data = [
    (1, "Arjun Patel", "Male", 30, 60000, "Aarav Sharma"),
    (2, "Aarav Sharma", "Male", 28, 55000, "Zara Singh"),
    (3, "Zara Singh", "Female", 35, 70000, "Arjun Patel"),
    (4, "Priya Reddy", "Female", 32, 65000, "Aarav Sharma"),
    (1, "Arjun Patel", "Male", 30, 60000, "Aarav Sharma"),
    (6, "Naina Verma", "Female", 31, 72000, "Arjun Patel"),
    (1, "Arjun Patel", "Male", 30, 60000, "Aarav Sharma"),
    (4, "Priya Reddy", "Female", 32, 65000, "Aarav Sharma"),
    (5, "Aditya Kapoor", "Male", 28, 58000, "Zara Singh"),
    (10, "Anaya Joshi", "Female", 27, 59000, "Aarav Sharma"),
    (11, "Rohan Malhotra", "Male", 36, 73000, "Zara Singh"),
    (3, "Zara Singh", "Female", 35, 70000, "Arjun Patel")
]

In [0]:
emp_df = spark.createDataFrame(data, schema)

In [0]:
emp_df.show()

+------+--------------+----------+-------+----------+------------+
|emp_id|      emp_name|emp_gender|emp_age|emp_salary| emp_manager|
+------+--------------+----------+-------+----------+------------+
|     1|   Arjun Patel|      Male|     30|     60000|Aarav Sharma|
|     2|  Aarav Sharma|      Male|     28|     55000|  Zara Singh|
|     3|    Zara Singh|    Female|     35|     70000| Arjun Patel|
|     4|   Priya Reddy|    Female|     32|     65000|Aarav Sharma|
|     1|   Arjun Patel|      Male|     30|     60000|Aarav Sharma|
|     6|   Naina Verma|    Female|     31|     72000| Arjun Patel|
|     1|   Arjun Patel|      Male|     30|     60000|Aarav Sharma|
|     4|   Priya Reddy|    Female|     32|     65000|Aarav Sharma|
|     5| Aditya Kapoor|      Male|     28|     58000|  Zara Singh|
|    10|   Anaya Joshi|    Female|     27|     59000|Aarav Sharma|
|    11|Rohan Malhotra|      Male|     36|     73000|  Zara Singh|
|     3|    Zara Singh|    Female|     35|     70000| Arjun Pa

In [0]:
emp_df1 = emp_df.groupBy(col('emp_id')).agg(count(col('emp_id')).alias('occurence'))

emp_df2 = emp_df1.filter(col('occurence') > 1)

final_df = emp_df2.join(emp_df, on= 'emp_id')

final_df.distinct().show()

+------+---------+-----------+----------+-------+----------+------------+
|emp_id|occurence|   emp_name|emp_gender|emp_age|emp_salary| emp_manager|
+------+---------+-----------+----------+-------+----------+------------+
|     1|        3|Arjun Patel|      Male|     30|     60000|Aarav Sharma|
|     3|        2| Zara Singh|    Female|     35|     70000| Arjun Patel|
|     4|        2|Priya Reddy|    Female|     32|     65000|Aarav Sharma|
+------+---------+-----------+----------+-------+----------+------------+



In [0]:
emp_df2.show()

+------+---------+
|emp_id|occurence|
+------+---------+
|     1|        3|
|     3|        2|
|     4|        2|
+------+---------+



In [0]:
emp_dfgroup = emp_df.groupBy(emp_df.columns).count()
emp_dfgroup.show()

+------+--------------+----------+-------+----------+------------+-----+
|emp_id|      emp_name|emp_gender|emp_age|emp_salary| emp_manager|count|
+------+--------------+----------+-------+----------+------------+-----+
|     2|  Aarav Sharma|      Male|     28|     55000|  Zara Singh|    1|
|     1|   Arjun Patel|      Male|     30|     60000|Aarav Sharma|    3|
|     3|    Zara Singh|    Female|     35|     70000| Arjun Patel|    2|
|     4|   Priya Reddy|    Female|     32|     65000|Aarav Sharma|    2|
|     6|   Naina Verma|    Female|     31|     72000| Arjun Patel|    1|
|     5| Aditya Kapoor|      Male|     28|     58000|  Zara Singh|    1|
|    11|Rohan Malhotra|      Male|     36|     73000|  Zara Singh|    1|
|    10|   Anaya Joshi|    Female|     27|     59000|Aarav Sharma|    1|
+------+--------------+----------+-------+----------+------------+-----+



In [0]:
final_df1 = emp_dfgroup.where(col("count") > 1)
final_df1.drop(col('count')).show()

+------+-----------+----------+-------+----------+------------+
|emp_id|   emp_name|emp_gender|emp_age|emp_salary| emp_manager|
+------+-----------+----------+-------+----------+------------+
|     1|Arjun Patel|      Male|     30|     60000|Aarav Sharma|
|     3| Zara Singh|    Female|     35|     70000| Arjun Patel|
|     4|Priya Reddy|    Female|     32|     65000|Aarav Sharma|
+------+-----------+----------+-------+----------+------------+



In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import count

In [0]:
partition = Window.partitionBy(emp_df.columns)

In [0]:
df_count = emp_df.withColumn("count", count('emp_id').over(partition))
df_count.where(col("count") > 1).distinct().drop(col('count')).show()

+------+-----------+----------+-------+----------+------------+
|emp_id|   emp_name|emp_gender|emp_age|emp_salary| emp_manager|
+------+-----------+----------+-------+----------+------------+
|     1|Arjun Patel|      Male|     30|     60000|Aarav Sharma|
|     3| Zara Singh|    Female|     35|     70000| Arjun Patel|
|     4|Priya Reddy|    Female|     32|     65000|Aarav Sharma|
+------+-----------+----------+-------+----------+------------+



In [0]:
df_count.distinct().show()

+------+--------------+----------+-------+----------+------------+-----+
|emp_id|      emp_name|emp_gender|emp_age|emp_salary| emp_manager|count|
+------+--------------+----------+-------+----------+------------+-----+
|     1|   Arjun Patel|      Male|     30|     60000|Aarav Sharma|    3|
|     2|  Aarav Sharma|      Male|     28|     55000|  Zara Singh|    1|
|     3|    Zara Singh|    Female|     35|     70000| Arjun Patel|    2|
|     4|   Priya Reddy|    Female|     32|     65000|Aarav Sharma|    2|
|     5| Aditya Kapoor|      Male|     28|     58000|  Zara Singh|    1|
|     6|   Naina Verma|    Female|     31|     72000| Arjun Patel|    1|
|    10|   Anaya Joshi|    Female|     27|     59000|Aarav Sharma|    1|
|    11|Rohan Malhotra|      Male|     36|     73000|  Zara Singh|    1|
+------+--------------+----------+-------+----------+------------+-----+



## Union

In [0]:
df1 = [(10, 20, 30)]
df2 = [(100, 200, 300)]
df1_schema = "col_0 int, col_1 int, col_2 int"
df2_schema = "col_1 int, col_2 int, col_3 int"
df_1 = spark.createDataFrame(df1, df1_schema)
df_2 = spark.createDataFrame(df2, df2_schema)

In [0]:
df_1.show()
df_2.show()

+-----+-----+-----+
|col_0|col_1|col_2|
+-----+-----+-----+
|   10|   20|   30|
+-----+-----+-----+

+-----+-----+-----+
|col_1|col_2|col_3|
+-----+-----+-----+
|  100|  200|  300|
+-----+-----+-----+



In [0]:
union_df = df_1.unionByName(df_2, allowMissingColumns=True)

In [0]:
union_df.show()

+-----+-----+-----+-----+
|col_0|col_1|col_2|col_3|
+-----+-----+-----+-----+
|   10|   20|   30| NULL|
| NULL|  100|  200|  300|
+-----+-----+-----+-----+



##rank dense rank row number

In [0]:
emp_data = [
    (101, "Varun", "Sales", 75000),
    (102, "Alia", "HR", 46000),
    (103, "David", "IT", 55000),
    (104, "Steve", "Sales", 75000),
    (105, "Soham", "HR", 46000),
    (106, "Kiron", "IT", 50000),
    (107, "Dhoni", "Sales", 68000),
    (108, "Tiger", "HR", 45000),
    (109, "Rock", "IT", 53000),
    (110, "Khali", "Sales", 75000)
]


In [0]:
emp_schema = StructType([
    StructField("EmployeeID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Department", StringType(), True),
    StructField("Salary", IntegerType(), True)
])

In [0]:
emp_df = spark.createDataFrame(emp_data, emp_schema)

emp_df.show()

+----------+-----+----------+------+
|EmployeeID| Name|Department|Salary|
+----------+-----+----------+------+
|       101|Varun|     Sales| 75000|
|       102| Alia|        HR| 46000|
|       103|David|        IT| 55000|
|       104|Steve|     Sales| 75000|
|       105|Soham|        HR| 46000|
|       106|Kiron|        IT| 50000|
|       107|Dhoni|     Sales| 68000|
|       108|Tiger|        HR| 45000|
|       109| Rock|        IT| 53000|
|       110|Khali|     Sales| 75000|
+----------+-----+----------+------+



In [0]:
#partition = Window.partitionBy(col('Department'))

from pyspark.sql.functions import row_number, dense_rank, rank, col, desc
windowDef = Window.orderBy(col("salary").desc())

In [0]:
emp_df1 = emp_df.withColumn("row_num", row_number().over(windowDef))\
    .withColumn("rank", rank().over(windowDef))\
        .withColumn("dense_rank", dense_rank().over(windowDef))

In [0]:
emp_df1.show()

+----------+-----+----------+------+-------+----+----------+
|EmployeeID| Name|Department|Salary|row_num|rank|dense_rank|
+----------+-----+----------+------+-------+----+----------+
|       101|Varun|     Sales| 75000|      1|   1|         1|
|       104|Steve|     Sales| 75000|      2|   1|         1|
|       110|Khali|     Sales| 75000|      3|   1|         1|
|       107|Dhoni|     Sales| 68000|      4|   4|         2|
|       103|David|        IT| 55000|      5|   5|         3|
|       109| Rock|        IT| 53000|      6|   6|         4|
|       106|Kiron|        IT| 50000|      7|   7|         5|
|       102| Alia|        HR| 46000|      8|   8|         6|
|       105|Soham|        HR| 46000|      9|   8|         6|
|       108|Tiger|        HR| 45000|     10|  10|         7|
+----------+-----+----------+------+-------+----+----------+



In [0]:
windowDef1 = Window.partitionBy(col('Department')).orderBy(col('Salary').desc())

wind_emp_df1 = emp_df.withColumn("row_num", row_number().over(windowDef1))\
    .withColumn("rank", rank().over(windowDef1))\
        .withColumn("dense_rank", dense_rank().over(windowDef1))

In [0]:
wind_emp_df1.show()

+----------+-----+----------+------+-------+----+----------+
|EmployeeID| Name|Department|Salary|row_num|rank|dense_rank|
+----------+-----+----------+------+-------+----+----------+
|       102| Alia|        HR| 46000|      1|   1|         1|
|       105|Soham|        HR| 46000|      2|   1|         1|
|       108|Tiger|        HR| 45000|      3|   3|         2|
|       103|David|        IT| 55000|      1|   1|         1|
|       109| Rock|        IT| 53000|      2|   2|         2|
|       106|Kiron|        IT| 50000|      3|   3|         3|
|       101|Varun|     Sales| 75000|      1|   1|         1|
|       104|Steve|     Sales| 75000|      2|   1|         1|
|       110|Khali|     Sales| 75000|      3|   1|         1|
|       107|Dhoni|     Sales| 68000|      4|   4|         2|
+----------+-----+----------+------+-------+----+----------+

